In [ ]:
from block_ai.lib.myblokus import point
from block_ai.lib.myblokus.point import Point
from block_ai.lib.myblokus.piece import Piece
from block_ai.lib.myblokus.corner import Corner
from block_ai.lib.myblokus.orientation import Orientation

import itertools
import numpy as np
import random

import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [ ]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.WARNING)

In [ ]:
"""
This class defines a Player as a collection of.

Pieces, valid moves, and invalid squares.
"""

from block_ai.lib.myblokus.piece import *
from block_ai.lib.myblokus.point import Point

import logging

class Player:

    def __init__(self, player_num):
        self.player_num = player_num
        self.pieces = gen_pieces()
        self.valid_moves = {}
        self.no_go_squares = []
        
    def update_state(self, move):
        logging.info("Deleting piece id: %s", move.piece_id)
        del self.pieces[move.piece_id]
        del self.valid_moves[move.corner]
        self.add_invalid_points(move)
        
        '''
        for c in self.valid_moves:
            try:
                del self.valid_moves[c][move.piece_id]
            except:
                pass
        self.clear_moves(move)
        '''

    def clear_moves(self, move):

        good_corners = list(self.valid_moves.keys())
        
        for corner in self.valid_moves.keys():
            if corner.p2 in invalid_points:
                good_corners.remove(corner)
                
        self.valid_moves = {c: self.valid_moves[c] for c in good_corners}
        
        for corner in self.valid_moves.keys():
            for old_m in self.get_corner_moves(corner):
                if self.overlap(old_m, move.get_footprint()):
                    self.valid_moves[corner][old_m.piece_id].remove(old_m)

    def add_invalid_points(self, move):
        rot = move.corner.get_rotation()
        
        invalid_points = [rot(p) for p in move.orientation.get_border_points()]
        invalid_points = list(filter(self.on_board, invalid_points))
        self.no_go_squares += invalid_points

    # TODO unduplicate this 
    def on_board(self, p):
        return p.x in range(20) and p.y in range(20)
        
    def is_move_valid(self, move):
        if not self.has_piece(move.piece_id):
            return False
        
        for p in move.orientation.points:
            if p in self.no_go_squares:
                logging.debug("Point: %s in player: %s no go squares", p, self.player_num)
                return False

        return True
        
    def is_orientation_valid(self, orientation):
        for p in orientation.points:
            if p in self.no_go_squares:
                
                return False
        return True
    
    def has_piece(self, piece_id):
        return piece_id in self.pieces
    
    def add_move(self, corner, move):
        try:
            corner_moves = self.valid_moves[corner]
            try:
                corner_moves[move.piece_id].append(move)
            except KeyError:
                corner_moves[move.piece_id] = [move]
        except KeyError:
            self.valid_moves[corner] = {}
            corner_moves = self.valid_moves[corner]
            corner_moves[move.piece_id] = [move]

    def delete_move(self, move):
        self.valid_moves[move.corner][move.piece_id].remove(move)

    def overlap(self, move, footprint):
        # TODO make this faster by sorting and incrementing
        for p in move.get_footprint():
            if p in footprint:
                return True
        return False

    def get_valid_moves(self):
        for corner in self.valid_moves:
            for m in self.get_corner_moves(corner):
                yield m

    def get_corners(self):
        return self.valid_moves.keys()
    
    def get_corner_moves(self, corner):
        for piece_id in self.valid_moves[corner]:
            for m in self.get_corner_piece_moves(corner, piece_id):
                yield m

    def get_corner_piece_moves(self, corner, piece_id):
        try:
            for m in self.valid_moves[corner][piece_id]:
                yield m
        except:
            pass
            
    def get_piece_moves(self, piece_id):
        for corner in self.valid_moves:
            try:
                for m in self.get_corner_piece_moves(corner, piece_id):
                    yield m
            except KeyError:
                continue

    def has_moves(self):
        return len(self.valid_moves) > 0
    
    def clean_valid_moves(self):
        good_corners = []
        for c in self.valid_moves.keys():
            if len(list(self.get_corner_moves(c))) > 0:
                good_corners.append(c)
                
        self.valid_moves = {c: self.valid_moves[c] for c in good_corners}
        
    
"""
    def clean_valid_moves(self):
        good_pieces = []
        for corner in self.valid_moves.keys():
            for piece_id in self.valid_moves[corner].keys():
                if len(self.valid_moves[corner][piece_id]) > 0:
                    good_pieces.append((corner, piece_id))
        
        self.valid_moves[corner] = {piece_id: self.valid_moves[c][p] for c, p in good_pieces}
            
        good_corners = []

        for corner in self.valid_moves.keys():
            if len(self.valid_moves[corner]) > 0:
                good_corners.append(corner, piece_id)

        self.valid_moves = {corner: self.valid_moves[corner] for corner in good_corners}
 """       

class Move:
    
    def __init__(self, orientation, player_id, piece_id, corner):
        self.orientation = orientation
        self.player_id = player_id
        self.piece_id = piece_id
        self.corner = corner

    def get_footprint(self):
        return [p for p in self.orientation.points]
    
    def __repr__(self):
        o_repr = repr(self.orientation)
        c_repr = repr(self.corner)
        return f"""Move(orientation={o_repr},
                        player_id={self.player_id},
                        piece_id='{self.piece_id}',
                        corner={c_repr})"""
    
    def __eq__(self, other):
        for attr in ["orientation", "player_id", "piece_id", "corner"]:
            if not eval(f"self.{attr} == other.{attr}"):
                return False
        return True

    def __str__(self):
        return f"Player ID: {self.player_id}\nPiece ID:{self.piece_id}\n{self.orientation}"

class Board:
    SIDE_LENGTH = 20
    EMPTY = -1

    def __init__(self):
        shape = (self.SIDE_LENGTH, self.SIDE_LENGTH)
        self.board = np.full(shape, self.EMPTY)
        self.player_pointer = 0
        self.players = [Player(i) for i in range(4)]
        self.set_starting_moves()
        self.move_history = []
    
    def set_starting_moves(self):
        logging.info("Starting Moves")
        corners = [Corner(Point(-1, -1), Point(0, 0)),
                   Corner(Point(-1, 20), Point(0, 19)),
                   Corner(Point(20, 20), Point(19, 19)),
                   Corner(Point(20, -1), Point(19, 0))]

        for i, c in enumerate(corners):
            self.add_corner_moves(c, i)

    def make_move(self, move):
        logging.debug("Making move: %s", move)
        if self.is_move_valid(move):
            self.update_state(move)
            self.move_history.append(move)
        else:
            logging.warning("Invalid move: %s", repr(move))

    def update_state(self, move):
        player = self.players[move.player_id]
        player.update_state(move)
        
        for p in move.get_footprint():
            self.assign(p, move.player_id)
            
        for corner in move.orientation.get_corners():
            logging.info("Adding corner: %s moves", corner)
            self.add_corner_moves(corner, move.player_id)
            
        # TODO make this better
        for m in self.get_players_moves(move.player_id):
            logging.debug("Evaluating Move: %s", move)
            if not self.is_move_valid(m):
                logging.debug("Deleting Move: %s", move)
                player.delete_move(m)
                
        player.clean_valid_moves()
            

    def add_corner_moves(self, corner, player_id):
        player = self.players[player_id]
        rotation = corner.get_rotation()

        for piece_id, piece in player.pieces.items():
            for orientation in piece.orientations:
                logging.debug("Examining orientation: %s", orientation)
                new_o = Orientation([corner.p2 + rotation(p) for p in orientation.points])
                logging.debug("new orientiation %s", new_o)
                m = Move(new_o, player_id, piece_id, corner)

                if self.is_move_valid(m):
                    logging.info("Adding Move: %s", indent(m, 1))
                    player.add_move(corner, m)

    def is_move_valid(self, move):
        if not self.are_squares_free(move.orientation):
            logging.debug(indent("Squares no free", 2))
            return False
        
        player = self.players[move.player_id]
        if not player.is_move_valid(move):
            logging.debug(indent("Player invalid", 2))
            return False
        
        return True

    def are_squares_free(self, orientation):
        for p in orientation.points:
            logging.debug(indent(f"Point: {p}", 2))
            if not self.is_square_free(p):
                return False
        return True

    def is_square_free(self, p):
        if not self.on_board(p):
            logging.debug(indent("Not on board", 2))
            return False
        if not self.point_empty(p):
            logging.debug(indent("point is not empty", 2))
            return False
        return True

    def on_board(self, point):
        valid_points = range(0, self.SIDE_LENGTH)
        return point.x in valid_points and point.y in valid_points
    
    def point_empty(self, p):
        val = self.check(p)
        logging.debug(val)
        return val == self.EMPTY
    
    def check(self, point):
        return self.board[point.y][point.x]
    
    def get_players_moves(self, player_id):
        return self.players[player_id].get_valid_moves()
    
    def assign(self, point, value):
        self.__assign(point, value, self.board)
        
    def __assign(self, point, value, board):
        if self.on_board(point):
            logging.debug("assigning %s: %s", point, value)
            board[point.y][point.x] = value
        else:
            logging.debug("%s off board not assigning", point)

    def __str__(self):
        return str(self.board)

    def copy(self):
        return eval(repr(self))

    def display(self, player_id=None, bad_move=None):
        self.__display(self.board, player_id, bad_move)
        
    def __display(self, board, player_id=None, bad_move=None):
        vis = BoardView()
        if player_id is not None:
            board = self.fill_player_pov(player_id)
            
        if bad_move is not None:
            board = self.fill_bad_move(board, bad_move)
        
        vis.display(board)

    def fill_player_pov(self, player_id):
        next_board = self.board.copy()
        adjacent_fill = 4
        corner_fill = 5
        for p in self.players[player_id].no_go_squares:
            self.__assign(p, adjacent_fill, next_board)
            
        for corner in self.players[player_id].valid_moves.keys():
            self.__assign(corner.p2, corner_fill, next_board)
        return next_board
    
    def fill_bad_move(self, board, move):
        bad_fill = 6
        for p in move.orientation.points:
            print(f"filling {p} with {bad_fill}")
            self.__assign(p, bad_fill, board)
        return board
            
def gen_pieces():
    pieces = {
        'p1': Piece([Point(0, 0)]),
        'p2': Piece([Point(0, 0), Point(0, 1)]),
        'p3': Piece([Point(0, 0), Point(0, 1), Point(0, 2)]),
        'p4': Piece([Point(0, 0), Point(0, 1), Point(1, 1)]),
        'p5': Piece([Point(0, 0), Point(1, 0), Point(0, 1), Point(1, 1)]),
        'p6': Piece([Point(0, 0), Point(1, 0), Point(2, 0), Point(3, 0)]),
        'p7': Piece([Point(0, 0), Point(1, 0), Point(1, 1), Point(2, 1)]),
        'p8': Piece([Point(0, 0), Point(1, 0), Point(1, 1), Point(2, 0)]),
        'p9': Piece([Point(0, 0), Point(1, 0), Point(1, 1), Point(2, 1)]),
        'p10': Piece([Point(0, 0), Point(1, 0), Point(2, 0), Point(3, 0), Point(4, 0)]),
        'p11': Piece([Point(0, 0), Point(1, 0), Point(2, 0), Point(3, 0), Point(3, 1)]),
        'p12': Piece([Point(0, 0), Point(1, 0), Point(2, 0), Point(2, 1), Point(3, 1)]),
        'p13': Piece([Point(0, 0), Point(1, 0), Point(2, 0), Point(1, 1), Point(2, 1)]),
        'p14': Piece([Point(0, 0), Point(1, 0), Point(2, 0), Point(0, 1), Point(2, 1)]),
        'p15': Piece([Point(0, 0), Point(1, 0), Point(2, 0), Point(3, 0), Point(2, 1)]),
        'p16': Piece([Point(0, 0), Point(1, 0), Point(2, 0), Point(1, 1), Point(1, 2)]),
        'p17': Piece([Point(0, 0), Point(1, 0), Point(2, 0), Point(2, 1), Point(2, 2)]),
        'p18': Piece([Point(0, 0), Point(1, 0), Point(1, 1), Point(2, 1), Point(2, 2)]),
        'p19': Piece([Point(0, 0), Point(0, 1), Point(1, 1), Point(2, 1), Point(2, 2)]),
        'p20': Piece([Point(0, 0), Point(1, 0), Point(1, 1), Point(2, 1), Point(1, 2)]),
        'p21': Piece([Point(0, 1), Point(1, 1), Point(1, 2), Point(1, 0), Point(2, 1)])
    }
    
    return pieces


def indent(obj, level):
    string = str(obj)
    join_str = "\n" + "\t" * level
    return "\t" * level + join_str.join(string.split("\n"))
     

## Random Agent

In [ ]:
def get_random_move(board, player_id):
    if board.players[player_id].has_moves():        
        moves = list(board.get_players_moves(player_id))
        rand_int = random.randint(0, len(moves))
        return moves[rand_int]
    else:
        raise StopIteration
        
def get_mistake(board, player_id):
    if board.players[player_id].has_moves():        
        moves = list(board.get_players_moves(player_id))
        for m in moves:
            if not board.is_move_valid(m):
                return m
    else:
        return None

def replay_n_moves(moves, n):
    b = Board()
    for move in moves[:n]:
        board.make_move(move)
    return board

def replay_game(moves):
    return replay_n_moves(moves, len(moves))
    
def play_bad_move(moves):
    logger.setLevel(logging.WARNING)
    board = replay_game(moves[:-1])
    board.show_player_view(0)
    logger.setLevel(logging.DEBUG)
    board.make_move(moves[-1])

def play_random_moves():
    player_id = 0
    board = Board()
    while board.players[player_id].has_moves():
        m = get_random_move(board, player_id)
        board.make_move(m)
        board.display()
    return board

def get_random_move(board, player_id):
    if board.players[player_id].has_moves():        
        moves = list(board.get_players_moves(player_id))
        rand_int = random.randint(0, len(moves))
        return moves[rand_int]
    else:
        raise StopIteration

def look_for_mistakes():
    player_id = 0
    board = Board()
    while board.players[player_id].has_moves():
        m = get_mistake(board, player_id)
        if m is None:
            m = get_random_move(board, player_id)
        else:
            print(repr(m))
            return board
        board.make_move(m)
        board.display()
    return board

## Status

Invalid Moves remain that are in no go squares

player.delete_move() works

maybe an update issue with no_go_squares

In [ ]:
moves = [
        Move(orientation=Orientation((Point(0, 0), Point(1, 0), Point(1, 1), Point(2, 1))),
             player_id=0,
             piece_id='p7',
             corner=Corner(p1=Point(-1, -1), p2=Point(0, 0))),

        Move(orientation=Orientation((Point(3, 2), Point(3, 3), Point(4, 0), Point(4, 1), Point(4, 2))),
             player_id=0,
             piece_id='p12',
             corner=Corner(p1=Point(2, 1), p2=Point(3, 2))),

        Move(orientation=Orientation((Point(5, 3), Point(6, 3), Point(6, 4), Point(6, 5), Point(6, 6))),
             player_id=0,
             piece_id='p11',
             corner=Corner(p1=Point(4, 2), p2=Point(5, 3)))
]

bad_move = Move(orientation=Orientation((Point(4, 4), Point(4, 5), Point(5, 5))),
                player_id=0,
                piece_id='p4',
                corner=Corner(p1=Point(3, 3), p2=Point(4, 4)))

## Visualization Code

In [ ]:
class BoardView:

    def __init__(self):
        pass
    
    def display(self, board):
        n = 20
        my_dpi = 20
        canvasSize = (n,n)
        fig,ax = self.createCanvas(canvasSize[0],canvasSize[1],my_dpi)
        color = "black"

        for row in range(n):
            for col in range(n):
                color = self.get_color(board[row][col])
                rect = self.createRectangle((1,1),(col,row),canvasSize,color)
                ax.add_patch(rect)

        plt.show()

    def createCanvas(self, width,height,my_dpi):
        fig,ax = plt.subplots(1)
        fig.dpi= my_dpi
        fig.set_size_inches(width,height)
        ax.set_xticks([])
        ax.set_yticks([])
        return fig,ax

    def createRectangle(self, wh, xy, cwch, color="black"):
        """
        All units are in inches. 

        Parameters
        ==========
        wh: (width,height) 
        xy: (x,y)
        cwch: (canvasWidth,canvasHeight)
        """
        return patches.Rectangle((xy[0]/cwch[0],xy[1]/cwch[1]),wh[0]/cwch[0],wh[1]/cwch[1],color=color)

    def get_color(self, val):
        mapping = {
            -1: "#DCDCDC",
            0: "yellow",
            1: "red",
            2: "green",
            3: "blue",
            4: "orange",
            5: "purple",
            6: "black"
        }
        return mapping[val]

In [ ]:
moves = [
        Move(orientation=Orientation((Point(0, 0), Point(1, 0), Point(1, 1), Point(2, 1))),
             player_id=0,
             piece_id='p7',
             corner=Corner(p1=Point(-1, -1), p2=Point(0, 0))),

        Move(orientation=Orientation((Point(3, 2), Point(3, 3), Point(4, 0), Point(4, 1), Point(4, 2))),
             player_id=0,
             piece_id='p12',
             corner=Corner(p1=Point(2, 1), p2=Point(3, 2))),

        Move(orientation=Orientation((Point(5, 3), Point(6, 3), Point(6, 4), Point(6, 5), Point(6, 6))),
             player_id=0,
             piece_id='p11',
             corner=Corner(p1=Point(4, 2), p2=Point(5, 3)))
]

bad_move = Move(orientation=Orientation((Point(4, 4), Point(4, 5), Point(5, 5))),
                player_id=0,
                piece_id='p4',
                corner=Corner(p1=Point(3, 3), p2=Point(4, 4)))

In [ ]:
m = Move(orientation=Orientation((Point(4, 4), Point(5, 4), Point(5, 5))),
                        player_id=0,
                        piece_id='p4',
                        corner=Corner(p1=Point(3, 3), p2=Point(4, 4)))

b1.players[0].delete_move(m)

In [ ]:
for m in b1.players[0].valid_moves[Corner(p1=Point(3, 3), p2=Point(4, 4))]['p4']:
    print(b1.is_move_valid(m))
    print(repr(m))

In [ ]:
logger.setLevel(logging.WARNING)

In [ ]:
b1 = Board()

In [ ]:
b1.make_move(moves[0])

In [ ]:
b1.display(0)

In [ ]:
b1.players[0].valid_moves[c]

In [ ]:
b1.make_move(moves[1])

In [ ]:
b1.display(0)

In [ ]:
bm = Move(orientation=Orientation((Point(4, 4), Point(4, 5), Point(5, 5))),
                player_id=0,
                piece_id='p4',
                corner=Corner(p1=Point(3, 3), p2=Point(4, 4)))

bm in b1.get_players_moves(0)

In [ ]:
moves = [
        Move(orientation=Orientation((Point(0, 0), Point(1, 0), Point(1, 1), Point(2, 1))),
             player_id=0,
             piece_id='p7',
             corner=Corner(p1=Point(-1, -1), p2=Point(0, 0))),

        Move(orientation=Orientation((Point(3, 2), Point(3, 3), Point(4, 0), Point(4, 1), Point(4, 2))),
             player_id=0,
             piece_id='p12',
             corner=Corner(p1=Point(2, 1), p2=Point(3, 2))),

        Move(orientation=Orientation((Point(5, 3), Point(6, 3), Point(6, 4), Point(6, 5), Point(6, 6))),
             player_id=0,
             piece_id='p11',
             corner=Corner(p1=Point(4, 2), p2=Point(5, 3)))
]

bad_move = Move(orientation=Orientation((Point(4, 4), Point(4, 5), Point(5, 5))),
                player_id=0,
                piece_id='p4',
                corner=Corner(p1=Point(3, 3), p2=Point(4, 4)))

In [ ]:

b1.players[0].valid_moves[c]

In [ ]:
replay = Board()
for m in moves:
    replay.make_move(m)
    replay.display(0)

In [ ]:
for c in replay.players[0].get_corners():
    print(repr(c))

In [ ]:

bad_move = Move(orientation=Orientation((Point(5, 3), Point(6, 3), Point(6, 4), Point(6, 5), Point(6, 6))),
             player_id=0,
             piece_id='p11',
             corner=Corner(p1=Point(4, 2), p2=Point(5, 3)))

replay.display(0, bad_move)

In [ ]:
board.display(False, 0)

In [ ]:
board = play_random_moves()

In [ ]:
board.display(False, 0)

In [ ]:
test_corner = Corner(Point(2, 1), Point(1, 2))

In [ ]:
for m in board.players[0].valid_moves.keys():
    print(m)

## Status
This code is close to working and is ugly as fuck

#### Questions:

why does it allow invalid moves in the below loop?

Theres an average branching factor of around 25
so call it 75 is that doable?





Number of chess game one guy says with 40 avg moves avg 30 moves per game
~ 10 ** 120 

James Hardy says (10 ** 50) ** 50 

Number of blockus game???

In [ ]:
f"{len(str(75 ** (20 * 4)))}"